# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,17.72,99,100,4.28,SH,1703907496
1,1,mahina,-17.5065,-149.4890,29.87,74,40,4.12,PF,1703907497
2,2,saipan,15.1355,145.7010,29.39,70,40,8.75,MP,1703907497
3,3,lebu,-37.6167,-73.6500,12.82,89,2,5.31,CL,1703907497
4,4,grytviken,-54.2811,-36.5092,-0.54,76,79,3.17,GS,1703907497


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display


# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    scale = 0.65,
    color = 'City')
# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal= city_data_df.drop(city_data_df[(city_data_df['Max Temp'] <= 18) & (city_data_df['Max Temp'] <= 25)].index, inplace=False)

# Drop any rows with null values
ideal.dropna(how='any', inplace=True)

# Display sample data
ideal.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,mahina,-17.5065,-149.4890,29.87,74,40,4.12,PF,1703907497
2,2,saipan,15.1355,145.7010,29.39,70,40,8.75,MP,1703907497
6,6,port elizabeth,-33.9180,25.5701,19.62,87,0,7.20,ZA,1703907498
8,8,flying fish cove,-10.4217,105.6791,28.82,78,75,6.69,CX,1703907498
19,19,waingapu,-9.6567,120.2641,31.95,60,28,4.28,ID,1703907501
21,21,quelimane,-17.8786,36.8883,23.97,94,75,2.06,MZ,1703907502
22,22,kununurra,-15.7667,128.7333,38.99,37,72,2.57,AU,1703907259
25,25,dwarka,22.2394,68.9678,23.56,51,0,2.84,IN,1703907503
29,29,wailua homesteads,22.0669,-159.3780,25.24,82,33,0.45,US,1703907503
30,30,west island,-12.1568,96.8225,28.99,74,100,5.14,CC,1703907504


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
hotel_df = ideal[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,mahina,PF,-17.5065,-149.4890,74,
2,saipan,MP,15.1355,145.7010,70,
6,port elizabeth,ZA,-33.9180,25.5701,87,
8,flying fish cove,CX,-10.4217,105.6791,78,
19,waingapu,ID,-9.6567,120.2641,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {"filter":"",
          "bias":"",
          "categories":"accommodation.hotel",
          "apiKey":geoapify_key}
    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mahina - nearest hotel: Motu Martin
saipan - nearest hotel: Chalan Kanoa Beach Hotel
port elizabeth - nearest hotel: Waterford Hotel
flying fish cove - nearest hotel: Christmas Island Lodge
waingapu - nearest hotel: HOTEL SANDLEWOOD
quelimane - nearest hotel: Mabassa
kununurra - nearest hotel: No hotel found
dwarka - nearest hotel: The Dwarika Hotel
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
west island - nearest hotel: Cocos Village Bungalows
hawaiian paradise park - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
karratha - nearest hotel: Karratha International Hotel
bandarbeyla - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
curumani - nearest hotel: Hotel Mayumir
port alfred - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
port mathurin - nearest hotel: Escale Vacances
moro - nearest hotel: No hotel found
kayangel - nearest hotel: No hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
1,mahina,PF,-17.5065,-149.4890,74,Motu Martin
2,saipan,MP,15.1355,145.7010,70,Chalan Kanoa Beach Hotel
6,port elizabeth,ZA,-33.9180,25.5701,87,Waterford Hotel
8,flying fish cove,CX,-10.4217,105.6791,78,Christmas Island Lodge
19,waingapu,ID,-9.6567,120.2641,60,HOTEL SANDLEWOOD
...,...,...,...,...,...,...
556,n'zeto,AO,-7.2312,12.8666,85,No hotel found
557,rio preto da eva,BR,-2.6994,-59.6997,98,Hotel Santo Antonio
558,soio,AO,-6.1349,12.3689,87,Hotel Maria Teres
560,burutu,NG,5.3509,5.5076,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 600,
    hover_cols = ["Hotel Name","Country"],
    size = 'Humidity',
    scale = 0.65,
    color = 'City')
# Display the map
map2
 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)